In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from skimage import io, img_as_float, img_as_ubyte
from skimage.transform import resize
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd
import gc
import random

# 시드 설정
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False



# DnCNN 모델 정의
class DnCNN(nn.Module):
    def __init__(self, channels=1, num_of_layers=17):
        super(DnCNN, self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        layers = []
        layers.append(nn.Conv2d(in_channels=channels, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
        layers.append(nn.ReLU(inplace=True))
        for _ in range(num_of_layers - 2):
            layers.append(nn.Conv2d(in_channels=features, out_channels=features, kernel_size=kernel_size, padding=padding, bias=False))
            layers.append(nn.BatchNorm2d(features))
            layers.append(nn.ReLU(inplace=True))
        layers.append(nn.Conv2d(in_channels=features, out_channels=channels, kernel_size=kernel_size, padding=padding, bias=False))
        self.dncnn = nn.Sequential(*layers)

    def forward(self, x):
        out = self.dncnn(x)
        return x - out

# 데이터셋 클래스 정의
class NoisyImageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0])
        image = img_as_float(io.imread(img_name))
        image = resize(image, (224, 224), anti_aliasing=True)
        noise = np.random.normal(0, 25/255, image.shape)
        noisy_image = image + noise
        noisy_image = np.clip(noisy_image, 0, 1)
        
        if self.transform:
            noisy_image = self.transform(noisy_image)
            image = self.transform(image)
        
        return noisy_image, image

# 모델 학습 함수 정의
def train_model(model, dataloader, criterion, optimizer, device, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for noisy_images, clean_images in tqdm(dataloader):
            noisy_images = noisy_images.permute(0, 3, 1, 2).float().to(device)
            clean_images = clean_images.permute(0, 3, 1, 2).float().to(device)
            
            outputs = model(noisy_images)
            loss = criterion(outputs, clean_images)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * noisy_images.size(0)
        
        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

# 모델 디노이징 함수 정의
def denoise_image(model, image, device):
    image = img_as_float(image)
    image = resize(image, (224, 224), anti_aliasing=True)
    input_tensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float().to(device)
    with torch.no_grad():
        output_tensor = model(input_tensor)
    output_image = output_tensor.squeeze(0).permute(1, 2, 0).cpu().numpy()
    output_image = np.clip(output_image, 0, 1)
    output_image = img_as_ubyte(output_image)
    return output_image

# 디바이스 설정 (GPU 사용 여부)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# 데이터셋 및 데이터로더 준비
csv_file = 'cv_challenge/train_augmented_byteam_rf_and_original.csv'
root_dir = 'cv_challenge/data_argumentation_byteam_rotate_flipped'
dataset = NoisyImageDataset(csv_file=csv_file, root_dir=root_dir)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# 모델, 손실 함수 및 옵티마이저 설정
model = DnCNN(channels=3).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습
train_model(model, dataloader, criterion, optimizer, device, num_epochs=5)

# 디노이징 적용
test_image_folder = "cv_challenge/test"
output_image_folder = "cv_challenge/denoised_images"

if not os.path.exists(output_image_folder):
    os.makedirs(output_image_folder)

test_images = [f for f in os.listdir(test_image_folder) if os.path.isfile(os.path.join(test_image_folder, f))]

for image_name in test_images:
    image_path = os.path.join(test_image_folder, image_name)
    image = io.imread(image_path)
    
    denoised_image = denoise_image(model, image, device)
    
    output_image_path = os.path.join(output_image_folder, image_name)
    io.imsave(output_image_path, denoised_image)
    print(f"Denoised image saved to {output_image_path}")

100%|██████████| 6280/6280 [1:22:57<00:00,  1.26it/s]


Epoch 1/5, Loss: 0.0043


100%|██████████| 6280/6280 [1:45:37<00:00,  1.01s/it]   


Epoch 2/5, Loss: 0.0007


100%|██████████| 6280/6280 [1:28:37<00:00,  1.18it/s]


...

Denoised image saved to cv_challenge/denoised_images\d5e33916cabf4119.jpg
Denoised image saved to cv_challenge/denoised_images\d5e71606bd76ed2c.jpg
Denoised image saved to cv_challenge/denoised_images\d608074b218acdf2.jpg
Denoised image saved to cv_challenge/denoised_images\d60a801df05f9b4f.jpg
Denoised image saved to cv_challenge/denoised_images\d60b72e1c9ce1a18.jpg
Denoised image saved to cv_challenge/denoised_images\d614c5c2cfed161c.jpg
Denoised image saved to cv_challenge/denoised_images\d6629b05f9490d9e.jpg
Denoised image saved to cv_challenge/denoised_images\d666a3a2fbcced6f.jpg
Denoised image saved to cv_challenge/denoised_images\d67cdf219830e13d.jpg
Denoised image saved to cv_challenge/denoised_images\d68f59641797cb15.jpg
Denoised image saved to cv_challenge/denoised_images\d68fefb4ab624bbf.jpg
Denoised image saved to cv_challenge/denoised_images\d6b6b9574a76758c.jpg
Denoised image saved to cv_challenge/denoised_images\d6bf9861273c09e5.jpg
Denoised image saved to cv_challenge/d

Denoised image saved to cv_challenge/denoised_images\de134b8d8b45f802.jpg
Denoised image saved to cv_challenge/denoised_images\de28c1c4f8c81c57.jpg
Denoised image saved to cv_challenge/denoised_images\de482dba4a789367.jpg
Denoised image saved to cv_challenge/denoised_images\de4a6b1984258913.jpg
Denoised image saved to cv_challenge/denoised_images\de4f81816aefede9.jpg
Denoised image saved to cv_challenge/denoised_images\de6ecb4369bfa015.jpg
Denoised image saved to cv_challenge/denoised_images\de7773a15386ece8.jpg
Denoised image saved to cv_challenge/denoised_images\de7abfe83d376417.jpg
Denoised image saved to cv_challenge/denoised_images\de93690927da7f29.jpg
Denoised image saved to cv_challenge/denoised_images\dec1013c70ea785f.jpg
Denoised image saved to cv_challenge/denoised_images\dede84fc2fd81e42.jpg
Denoised image saved to cv_challenge/denoised_images\dedfd817bb5bb34b.jpg
Denoised image saved to cv_challenge/denoised_images\df0b8b24c44c0dca.jpg
Denoised image saved to cv_challenge/d

Denoised image saved to cv_challenge/denoised_images\e66c61a04f1b4c06.jpg
Denoised image saved to cv_challenge/denoised_images\e68bc3bdeb2d2c50.jpg
Denoised image saved to cv_challenge/denoised_images\e69911daeb007e51.jpg
Denoised image saved to cv_challenge/denoised_images\e6a109b3b95d9bf4.jpg
Denoised image saved to cv_challenge/denoised_images\e6cda492548183c8.jpg
Denoised image saved to cv_challenge/denoised_images\e6f07cad96de721b.jpg
Denoised image saved to cv_challenge/denoised_images\e706792154346dc3.jpg
Denoised image saved to cv_challenge/denoised_images\e75f65cb5e1c3a57.jpg
Denoised image saved to cv_challenge/denoised_images\e76da7e84ff52898.jpg
Denoised image saved to cv_challenge/denoised_images\e7791924b7a8c5b1.jpg
Denoised image saved to cv_challenge/denoised_images\e7870b81d299c64d.jpg
Denoised image saved to cv_challenge/denoised_images\e7b69abf73fdb944.jpg
Denoised image saved to cv_challenge/denoised_images\e7c1e8cb2d86c63d.jpg
Denoised image saved to cv_challenge/d

Denoised image saved to cv_challenge/denoised_images\f00899924afb0d57.jpg
Denoised image saved to cv_challenge/denoised_images\f0231abdaf73d007.jpg
Denoised image saved to cv_challenge/denoised_images\f02840648bee247d.jpg
Denoised image saved to cv_challenge/denoised_images\f04b1c2e6ec6b359.jpg
Denoised image saved to cv_challenge/denoised_images\f06fefb3abfc9877.jpg
Denoised image saved to cv_challenge/denoised_images\f08aba0820e3ced2.jpg
Denoised image saved to cv_challenge/denoised_images\f08dd09abe9f17a5.jpg
Denoised image saved to cv_challenge/denoised_images\f0c5a628e887deae.jpg
Denoised image saved to cv_challenge/denoised_images\f0c99030ede49938.jpg
Denoised image saved to cv_challenge/denoised_images\f0cb4c15818227fd.jpg
Denoised image saved to cv_challenge/denoised_images\f0d149ab498b07df.jpg
Denoised image saved to cv_challenge/denoised_images\f0db66f90ef8b655.jpg
Denoised image saved to cv_challenge/denoised_images\f0f8f8cae2aa4f91.jpg
Denoised image saved to cv_challenge/d

Denoised image saved to cv_challenge/denoised_images\f867719ca398753f.jpg
Denoised image saved to cv_challenge/denoised_images\f86bf07a146f6d3d.jpg
Denoised image saved to cv_challenge/denoised_images\f87a2f7db038dca5.jpg
Denoised image saved to cv_challenge/denoised_images\f87fdd6c08a4fb1f.jpg
Denoised image saved to cv_challenge/denoised_images\f8a3a3bffd832fb0.jpg
Denoised image saved to cv_challenge/denoised_images\f8b8397d810afe6f.jpg
Denoised image saved to cv_challenge/denoised_images\f8b923cc784b1a01.jpg
Denoised image saved to cv_challenge/denoised_images\f8d1c4541dbb4e9b.jpg
Denoised image saved to cv_challenge/denoised_images\f8e1fb97fa382075.jpg
Denoised image saved to cv_challenge/denoised_images\f8ea5351245901e1.jpg
Denoised image saved to cv_challenge/denoised_images\f915c7ed2f0da1f0.jpg
Denoised image saved to cv_challenge/denoised_images\f92e9dbad6f08b68.jpg
Denoised image saved to cv_challenge/denoised_images\f938a5eacf2ac36e.jpg
Denoised image saved to cv_challenge/d

In [3]:
torch.save(model.state_dict(), 'CuDNN_5.pth')